##### 전처리

In [ ]:
import numpy as np
import pandas as pd
import re
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
 # load csv
users = pd.read_csv("../data/users.csv")
books = pd.read_csv("../data/books.csv")
test = pd.read_csv("../data/test_ratings.csv")
train = pd.read_csv("../data/train_ratings.csv")
train_counts = train.groupby('user_id')['rating'].count().sort_values(ascending=False)

 # location 전처리
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

# publisher 전처리
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df = pd.DataFrame(publisher_dict.items(),columns = ['publisher','count'])
publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)

modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values

books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
books['category'] = books['category'].str.lower()
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
books['category'] = books['category'].str.lower()

In [ ]:
## 유저와 출판사 관계

In [ ]:
 # isbn을 기준으로 train과 books merge
merge1 = train.merge(books, how='left', on='isbn')
data = merge1.merge(users, how='inner', on='user_id')
print('merge 결과 shape: ', data.shape)

In [ ]:
data.columns

In [ ]:
 # ['user_id', 'publisher']로 그룹화 한뒤 유저별로 출판사마다 isbn의 개수를 세주었다.
user_publisher_interactions = pd.DataFrame(data.groupby(['user_id', 'publisher'])['isbn'].count())
user_publisher_interactions = user_publisher_interactions.reset_index()

In [ ]:
user_publisher_interactions

In [ ]:
user_publisher_interactions[(2 <= user_publisher_interactions['isbn']) & (user_publisher_interactions['isbn'] <  10)]

In [ ]:
# boolean 인덱스로 해당 출판사에서 i권 이상 i + 1 권이하의 책을 구매한 합을 각각 repurchase에 저장한다.
repurchase = []
for i in range(2, 20):
    repurchase.append(user_publisher_interactions[(i <= user_publisher_interactions['isbn']) & (user_publisher_interactions['isbn'] < i + 1)]['publisher'].count())
repurchase = np.array(repurchase)

In [ ]:
x = np.arange(len(repurchase))
number = [i+2 for i in x]
repurchase_user_cnt = repurchase

plt.bar(x, repurchase_user_cnt)
plt.xticks(x, number)

plt.show

In [ ]:
sum(repurchase)

In [ ]:
user_publisher_interactions['publisher'].count()

In [ ]:
sum(repurchase)/user_publisher_interactions['publisher'].count()

In [ ]:
유저는 15프로 확률로 구매했던 출판사에서 책을 구매한다.(?)